In [1]:
import pickle
import os
import numpy as np
import json
# 读取.pkl文件
# with open(r'../output/val_1/val_1_vitcliplarge-base-yolov8-bestacc/2023-11-02-15-38-30/val_1_vitcliplarge-base-yolov8-bestacc.pkl', 'rb') as f:
with open(r'submit-require-task1.pkl', 'rb') as f:
    data = pickle.load(f)

In [2]:
gt_file = '/data/llk/projects/road-r/road-dataset/road/road_trainval_v1.0.json'
f = open(gt_file, "r")
annojson = json.loads(f.read())
f.close()
annojson.keys()

dict_keys(['all_input_labels', 'all_av_action_labels', 'av_action_labels', 'agent_labels', 'action_labels', 'duplex_labels', 'triplet_labels', 'loc_labels', 'old_loc_labels', 'db', 'label_types', 'all_duplex_labels', 'all_triplet_labels', 'all_agent_labels', 'all_loc_labels', 'all_action_labels', 'duplex_childs', 'triplet_childs'])

In [3]:
def is_part_of_subsets(split_ids, SUBSETS):
    is_it = False
    for subset in SUBSETS:
        if subset in split_ids:
            is_it = True
    return is_it
task1_labels = ["2014-07-14-14-49-50_stereo_centre_01","2015-02-03-19-43-11_stereo_centre_04","2015-02-24-12-32-19_stereo_centre_04"]

In [4]:
all_video_names = annojson["db"].keys()
val_1_video_names = []
for vname in all_video_names:
    if is_part_of_subsets(annojson["db"][vname]["split_ids"], ["val_1"]) and vname not in task1_labels:
        val_1_video_names.append(vname)

In [5]:
all_video_names = annojson["db"].keys()
train_1_video_names = []
for vname in all_video_names:
    if is_part_of_subsets(annojson["db"][vname]["split_ids"], ["train_1"]) and vname not in task1_labels:
        train_1_video_names.append(vname)

In [6]:
annojson['all_agent_labels']

['Ped',
 'Car',
 'Cyc',
 'Mobike',
 'SmalVeh',
 'MedVeh',
 'LarVeh',
 'Bus',
 'EmVeh',
 'TL',
 'OthTL']

In [7]:
from tqdm import tqdm
num_classes = 41
all_det_info = [{} for i in range(num_classes)]
frame_id = 0
action_and_location_conf_thresh = 0.00
for video_name in tqdm(val_1_video_names):
    for frame_key in sorted(data[video_name].keys(), key=lambda k:int(k.split(".")[0])):
        curframe_obj = data[video_name][frame_key]
        for tube_idx in range(len(curframe_obj)):
            x1, y1, x2, y2 = curframe_obj[tube_idx]["bbox"]
            labels_conf = curframe_obj[tube_idx]["labels"]
            # if max(labels_conf) < 0.5:
            #     continue
            for cls_id in range(num_classes):
                if frame_id not in all_det_info[cls_id].keys():
                    all_det_info[cls_id][frame_id] = []
                all_det_info[cls_id][frame_id].append([x1, y1, x2, y2,  0 if cls_id >= 10 and labels_conf[cls_id] < action_and_location_conf_thresh else labels_conf[cls_id]])
        for cls_id in range(num_classes):
            if frame_id in all_det_info[cls_id].keys():
                all_det_info[cls_id][frame_id] = np.array(all_det_info[cls_id][frame_id])
        frame_id += 1

100%|██████████| 3/3 [00:05<00:00,  1.70s/it]


In [8]:
frame_id = 0
all_gt_info = {}
w, h = 1280, 960
def update_agent_clsid(agent_id):
    agent_name = annojson['all_agent_labels'][agent_id]
    if agent_name in annojson['agent_labels']:
        agent_id = annojson['agent_labels'].index(agent_name)
    else:
        return -1
    return agent_id
    # if agent_id == 4:
    #     return -1
    # if agent_id > 4:
    #     agent_id -= 1
    # return agent_id

def update_action_clsid(action_id):
    action_name = annojson['all_action_labels'][action_id]
    if action_name in annojson['action_labels']:
        action_id = annojson['action_labels'].index(action_name)
    else:
        return -1
    return action_id + 10

def update_location_clsid(location_id):
    return location_id + 29

for video_name in tqdm(val_1_video_names):
    for frame_key in sorted(annojson["db"][video_name]["frames"].keys(), key=lambda k:int(k)):
        curframe_obj = annojson["db"][video_name]["frames"][frame_key]
        if frame_id not in all_gt_info.keys():
            all_gt_info[frame_id] = []
        if "annos" in curframe_obj.keys():
            if len(curframe_obj["annos"]) > 0:
                for tube_k in curframe_obj["annos"].keys():
                    tube_item = curframe_obj["annos"][tube_k]
                    x1, y1, x2, y2 = tube_item['box']
                    x1, y1, x2, y2 = x1*w, y1*h, x2*w, y2*h
                    for agent_id in tube_item["agent_ids"]:
                        new_agent_id = update_agent_clsid(agent_id)
                        if new_agent_id == -1:
                            continue
                        all_gt_info[frame_id].append([x1, y1, x2, y2, new_agent_id])
                    for action_id in tube_item["action_ids"]:
                        new_action_id = update_action_clsid(action_id)
                        if new_action_id == -1:
                            continue
                        all_gt_info[frame_id].append([x1, y1, x2, y2, new_action_id])
                    for location_id in tube_item["loc_ids"]:
                        new_location_id = update_location_clsid(location_id)
                        if new_location_id == -1:
                            continue
                        all_gt_info[frame_id].append([x1, y1, x2, y2, new_location_id])
            else:
                all_gt_info[frame_id] = []
        else:
            all_gt_info[frame_id] = []
        all_gt_info[frame_id] = np.array(all_gt_info[frame_id])
        frame_id += 1

100%|██████████| 3/3 [00:00<00:00, 10.19it/s]


## 计算mAP的代码 - baseline版本

In [9]:
def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap*100

def compute_iou(box, cls_gt_boxes):

    ious = np.zeros(cls_gt_boxes.shape[0])

    for m in range(cls_gt_boxes.shape[0]):
        gtbox = cls_gt_boxes[m]

        xmin = max(gtbox[0], box[0])
        ymin = max(gtbox[1], box[1])
        xmax = min(gtbox[2], box[2])
        ymax = min(gtbox[3], box[3])
        iw = np.maximum(xmax - xmin, 0.)
        ih = np.maximum(ymax - ymin, 0.)
        if iw > 0 and ih > 0:
            intsc = iw*ih
        else:
            intsc = 0.0
        union = (gtbox[2] - gtbox[0]) * (gtbox[3] - gtbox[1]) + \
            (box[2] - box[0]) * (box[3] - box[1]) - intsc
        ious[m] = intsc/union

    return ious

def get_gt_of_cls(gt_boxes, cls):
    cls_gt_boxes = []
    for i in range(gt_boxes.shape[0]):
        if len(gt_boxes.shape) > 1 and int(gt_boxes[i, -1]) == cls:
            cls_gt_boxes.append(gt_boxes[i, :-1])
    return np.asarray(cls_gt_boxes)

def evaluate_detections(gt_boxes, det_boxes, classes=[], iou_thresh=0.5):
    ap_strs = []
    num_frames = len(gt_boxes)
    print('Evaluating for '+ str(num_frames) + ' frames')
    ap_all = np.zeros(len(classes), dtype=np.float32)
    # loop over each class 'cls'
    for cls_ind, class_name in enumerate(tqdm(classes)):
        scores = np.zeros(num_frames * 2000)
        istp = np.zeros(num_frames * 2000)
        det_count = 0
        num_postives = 0.0
        for nf in range(num_frames):  # loop over each frame 'nf'
                # if len(gt_boxes[nf])>0 and len(det_boxes[cls_ind][nf]):
            # get frame detections for class cls in nf
            if nf in det_boxes[cls_ind].keys():
                frame_det_boxes = np.copy(det_boxes[cls_ind][nf])
            else:
                frame_det_boxes = np.array([])
            # get gt boxes for class cls in nf frame
            cls_gt_boxes = get_gt_of_cls(np.copy(gt_boxes[nf]), cls_ind)
            num_postives += cls_gt_boxes.shape[0]
            # check if there are dection for class cls in nf frame
            if frame_det_boxes.shape[0] > 0:
                # sort in descending order
                sorted_ids = np.argsort(-frame_det_boxes[:, -1])
                for k in sorted_ids:  # start from best scoring detection of cls to end
                    box = frame_det_boxes[k, :-1]  # detection bounfing box
                    score = frame_det_boxes[k, -1]  # detection score
                    ispositive = False  # set ispostive to false every time
                    # we can only find a postive detection
                    if cls_gt_boxes.shape[0] > 0:
                        # if there is atleast one gt bounding for class cls is there in frame nf
                        # compute IOU between remaining gt boxes
                        iou = compute_iou(box, cls_gt_boxes)
                        # and detection boxes
                        # get the max IOU window gt index
                        maxid = np.argmax(iou)
                        # check is max IOU is greater than detection threshold
                        if iou[maxid] >= iou_thresh:
                            ispositive = True  # if yes then this is ture positive detection
                            # remove assigned gt box
                            cls_gt_boxes = np.delete(cls_gt_boxes, maxid, 0)
                    # fill score array with score of current detection
                    scores[det_count] = score
                    if ispositive:
                        # set current detection index (det_count)
                        istp[det_count] = 1
                        #  to 1 if it is true postive example
                    det_count += 1
        if num_postives < 1:
            num_postives = 1
        scores = scores[:det_count]
        istp = istp[:det_count]
        argsort_scores = np.argsort(-scores)  # sort in descending order
        istp = istp[argsort_scores]  # reorder istp's on score sorting
        fp = np.cumsum(istp == 0)  # get false positives
        tp = np.cumsum(istp == 1)  # get  true positives
        fp = fp.astype(np.float64)
        tp = tp.astype(np.float64)
        recall = tp / float(num_postives)  # compute recall
        # compute precision
        precision = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
        # compute average precision using voc2007 metric
        cls_ap = voc_ap(recall, precision)
        ap_all[cls_ind] = cls_ap
        ap_str = class_name + ' : ' + \
            str(num_postives) + ' : ' + str(det_count) + ' : ' + str(cls_ap)
        ap_strs.append(ap_str)

    mAP = np.mean(ap_all)
    print('mean ap '+ str(mAP))
    print("agent mAP: ", ap_all[:10].mean())
    print("action mAP: ", ap_all[10:29].mean())
    print("location mAP: ", ap_all[29:41].mean())
    return mAP, ap_all, ap_strs

In [10]:
def load_label_names(label_names_file):
    f = open(label_names_file, "r")
    labelnames = json.loads(f.read())
    
    agent_labels = labelnames["agent_names"]
    action_labels = labelnames["action_names"]
    loc_labels = labelnames["location_names"]
    f.close()
    return agent_labels, action_labels, loc_labels
label_names_file = "/data/llk/projects/road-r/yolov8/road-r-2023-yolov8-semi/configs/label_names.json"
agent_labels, action_labels, loc_labels = load_label_names(label_names_file)
class_names = []
class_names.extend(agent_labels)
class_names.extend(action_labels)
class_names.extend(loc_labels)

In [11]:
mAP, ap_all, ap_strs = evaluate_detections(gt_boxes=all_gt_info, det_boxes=all_det_info, classes=class_names, iou_thresh=0.5)

Evaluating for 18000 frames


100%|██████████| 41/41 [00:17<00:00,  2.31it/s]

mean ap 28.15609
agent mAP:  46.568386
action mAP:  20.461746
location mAP:  24.995216


In [12]:
f = open("./result.txt", "w", encoding="utf-8")
for line in ap_strs:
    print(line)
    line += '\n'
    f.write(line)
f.close()

Ped : 18949.0 : 54816 : 56.92677056431496
Car : 11032.0 : 54816 : 64.87879815204938
Cyc : 6139.0 : 54816 : 61.1618194800675
Mobike : 2121.0 : 54816 : 10.318326550873303
MedVeh : 9895.0 : 54816 : 39.1641410228933
LarVeh : 633.0 : 54816 : 3.8810643261972677
Bus : 1834.0 : 54816 : 80.38612658852958
EmVeh : 1 : 54816 : 0.0
TL : 7690.0 : 54816 : 72.90587123451755
OthTL : 1810.0 : 54816 : 76.06094572219311
Red : 6309.0 : 54816 : 61.18039491902191
Amber : 381.0 : 54816 : 67.25355374472953
Green : 2810.0 : 54816 : 34.638386780834495
MovAway : 14699.0 : 54816 : 39.2959294732644
MovTow : 13921.0 : 54816 : 44.21291283678131
Mov : 1587.0 : 54816 : 23.695880928611853
Brake : 1097.0 : 54816 : 12.963362862726935
Stop : 16895.0 : 54816 : 33.24517357878363
IncatLft : 1884.0 : 54816 : 4.689689573830523
IncatRht : 2015.0 : 54816 : 18.578067179741787
HazLit : 1000.0 : 54816 : 7.848621053555559
TurLft : 772.0 : 54816 : 5.575325177251311
TurRht : 704.0 : 54816 : 13.477214158347643
Ovtak : 202.0 : 54816 : 0.

In [13]:
print("agent mAP: ", ap_all[:10].mean())
print("action mAP: ", ap_all[10:29].mean())
print("location mAP: ", ap_all[29:41].mean())
mAP

agent mAP:  46.568386
action mAP:  20.461746
location mAP:  24.995216


28.15609